In [206]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices


### SETS

In [207]:
fuel_price_csv = 'fuel_price2_27s.csv'
# Velg antall av de ulike settene her
#set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 6)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 6)
tp2_values = range(2,7)
f_values = range(1, 7)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 2240)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, 19) # 18 installations
s_values = range(1,12) # 11 systems
w_values = range(1, 28) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6


In [208]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = '../run_model/SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'TP2 scrap': tp2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [209]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)
combinations_w = itertools.product(w_values)
combinations_r = itertools.product(r_values)

#### Inputdata

In [210]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.171, 2:0.163, 3:0.514, 4:0.328, 5:0.222, 6:0.297}




# fuel_dict = {
#     1: (0.178, 0.094, 'fossil'),      # MGO
#     2: (0.146, 0.073, 'fossil'),      # LNG
#     3: (0.491, 0.261, 'green_fuels'), # GREEN METHANOL
#     4: (0.324, 0.178, 'green_fuels'), # GREEN NH3
#     5: (0.278, 0.147, 'bio'),         # B30
#     6: (0.245, 0.130, 'bio'),         # B20
#     7: (0.228, 0.121, 'bio'),         # B15
#     8: (0.201, 0.106, 'bio'),         # B7
#     9: (0.301, 0.129, 'green_fuels')  # Helelektrisk
# }
 # Example fuel bounds with group information


# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000182, 2: 0.000153, 3: 0.000033, 4: 0.000033, 5: 0.000134, 6: 0.000150, 7: 0.000158, 8: 0.000170, 9: 0.000008 }
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0, 7: 0.0}
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw 


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {0: 0, 1: 0.50, 2: 0.55, 3: 0.65, 4: 0.70, 5: 0.80 } # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {0: 0, 1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

capacity_systems = {1: 100, 2: 100, 3: 100, 4: 100, 5: 70, 6: 70, 7: 70, 8: 70, 9: 90, 10: 90, 11: 70, 12: 90, 13: 90}

#### Lese inn data fra rutegenereringsprogrammet

In [211]:
mappenavn = 'generated_datafiles_2239'

# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation_mongstad/{mappenavn}/mj_route.csv')
#mj_route_el = pd.read_csv('../route_generation/generated_datafiles_reduced/mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv('../route_generation_mongstad/{mappenavn}/hours_route.csv')
#hours_route_el = pd.read_csv('../route_generation/generated_datafiles_reduced/hours_route_strøm.csv')

#demand_el = pd.read_csv('../route_generation/generated_datafiles_reduced/demand_strøm.csv')
demand = pd.read_csv('../route_generation_mongstad/{mappenavn}/demand.csv')

routes_file = "../route_generation_mongstad/{mappenavn}/routes.csv"
excel_file = '../run_model/Parameterdata.xlsx'
platform_demand = "../route_generation_mongstad/clustering/output_platforms_demand.csv"

# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = '../run_model/Parameterdata.xlsx'


FileNotFoundError: [Errno 2] No such file or directory: '../route_generation_mongstad/{mappenavn}/mj_route.csv'

##### Generere P_w

In [ ]:
df_probability = pd.DataFrame(combinations_w, columns=['O'])

df_probability['Value'] = 1/len(list(w_values))

df_probability.head()


,O,Value
0,1,0.037037
1,2,0.037037
2,3,0.037037
3,4,0.037037
4,5,0.037037


In [ ]:

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_probability.to_excel(writer, sheet_name='Probability', index=False)

In [ ]:
distance_dict = {}

for idx, value in enumerate(mj_route,1):
    distance_dict[idx] = value

print(distance_dict)

{1: 'mj_route'}


##### Distance dict for compatibility og cost of fuel

In [ ]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
print(distance_dict)
# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in enumerate(distance_dict.values())}
print(len(distance_dict))
print(distance_dict)


{0: 129060.0, 1: 150025.2, 2: 99089.6, 3: 163154.8, 4: 160114.8, 5: 178914.4, 6: 125450.4, 7: 151530.4, 8: 135416.8, 9: 114740.0, 10: 119480.0, 11: 114273.6, 12: 123060.0, 13: 168248.4, 14: 159874.8, 15: 82181.6, 16: 77230.0, 17: 71374.8, 18: 209565.2, 19: 192709.6, 20: 233014.8, 21: 236534.8, 22: 221734.4, 23: 169310.4, 24: 164030.4, 25: 198876.8, 26: 141160.0, 27: 139260.0, 28: 151653.6, 29: 134200.0, 30: 230988.4, 31: 225414.8, 32: 139081.6, 33: 142290.0, 34: 130994.8, 35: 186954.8, 36: 202300.0, 37: 207660.0, 38: 194299.6, 39: 152527.2, 40: 205327.2, 41: 156322.0, 42: 164525.2, 43: 197665.2, 44: 190458.8, 45: 201965.2, 46: 199485.2, 47: 194060.0, 48: 160138.4, 49: 156615.2, 50: 158760.0, 51: 194712.8, 52: 190712.8, 53: 138780.0, 54: 202300.0, 55: 149706.4, 56: 146149.6, 57: 178969.6, 58: 143191.6, 59: 160629.6, 60: 202378.0, 61: 192952.8, 62: 122711.2, 63: 114079.6, 64: 120692.8, 65: 198709.6, 66: 206869.2, 67: 169165.2, 68: 229565.2, 69: 200171.6, 70: 210854.8, 71: 220314.8, 72: 2

##### Compatibility RS

In [ ]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
demand_dict = demand['Demand'].to_dict()
#demand_dict = demand_dict['Demand']

# lager egne distanser for elektrisk

#specific_demand_dict = demand_el['Demand'].to_dict()
#specific_demand_dict= specific_demand_dict['Demand']

#electric_demand_dict = {key + 1: value for key, value in specific_demand_dict.items()}

demand_dict = {key + 1: value for key, value in demand_dict.items()}




In [ ]:
# Lage combinations for rs
# For hver rad med r s, sjekke om kapasiteten i s er større enn eller lik demanden til r. Sett til 1 hvis true, 0 ellers

# Create a DataFrame from the combinations
df_routecap = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Display the first few rows of the DataFrame
print(df_routecap.head())

df_routecap['Value'] = 0

# Iterate over the DataFrame
for index, row in df_routecap.iterrows():
    r = row['R']
    s = row['S']
    if demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1
    # if s == 11 and electric_demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
    #     df_routecap.at[index, 'Value'] = 1

#print(df_routecap)

   R  S
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [ ]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_routecap.to_excel(writer, sheet_name='Compatibility rs', index=False)

#### Demand for set covering

In [ ]:
df_demand = pd.DataFrame(list(demand_dict.items()), columns=['R', 'Value'])

print(df_demand)

         R  Value
0        1   18.0
1        2   25.0
2        3   40.0
3        4   29.0
4        5   29.0
...    ...    ...
2234  2235   93.0
2235  2236   98.5
2236  2237   98.5
2237  2238  100.0
2238  2239   98.5

[2239 rows x 2 columns]


In [ ]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand route', index=False)

##### Cost of fuel per route

In [ ]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
# #specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))

print(distance_dict)
#print(len(electric_distance_dict))

2239
{1: 129060.0, 2: 150025.2, 3: 99089.6, 4: 163154.8, 5: 160114.8, 6: 178914.4, 7: 125450.4, 8: 151530.4, 9: 135416.8, 10: 114740.0, 11: 119480.0, 12: 114273.6, 13: 123060.0, 14: 168248.4, 15: 159874.8, 16: 82181.6, 17: 77230.0, 18: 71374.8, 19: 209565.2, 20: 192709.6, 21: 233014.8, 22: 236534.8, 23: 221734.4, 24: 169310.4, 25: 164030.4, 26: 198876.8, 27: 141160.0, 28: 139260.0, 29: 151653.6, 30: 134200.0, 31: 230988.4, 32: 225414.8, 33: 139081.6, 34: 142290.0, 35: 130994.8, 36: 186954.8, 37: 202300.0, 38: 207660.0, 39: 194299.6, 40: 152527.2, 41: 205327.2, 42: 156322.0, 43: 164525.2, 44: 197665.2, 45: 190458.8, 46: 201965.2, 47: 199485.2, 48: 194060.0, 49: 160138.4, 50: 156615.2, 51: 158760.0, 52: 194712.8, 53: 190712.8, 54: 138780.0, 55: 202300.0, 56: 149706.4, 57: 146149.6, 58: 178969.6, 59: 143191.6, 60: 160629.6, 61: 202378.0, 62: 192952.8, 63: 122711.2, 64: 114079.6, 65: 120692.8, 66: 198709.6, 67: 206869.2, 68: 169165.2, 69: 229565.2, 70: 200171.6, 71: 210854.8, 72: 220314.8,

In [ ]:
# Convert dictionaries to DataFrames
df_distance = pd.DataFrame(distance_dict.items(), columns=['R', 'Value'])
#df_specific_distance = pd.DataFrame(electric_distance_dict.items(), columns=['R', 'Value'])

# Sort DataFrames by 'R' values
df_distance = df_distance.sort_values(by='R')
#df_specific_distance = df_specific_distance.sort_values(by='R')


# print("Distance Dictionary DataFrame:")
# print(df_distance)

# print("\nSpecific Distance Dictionary DataFrame:")
# print(df_specific_distance)

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_distance.to_excel(writer, sheet_name='Distance', index=False)

# with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     df_specific_distance.to_excel(writer, sheet_name='Specific distance', index=False)

Generere alle kombinasjoner av R og F

In [ ]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [ ]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
# def calculate_value(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return fuel_prices[row['F']] * distance

def calculate_value(row):
    distance = distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

# df_fuelprice

##### Fuelcost 2

In [ ]:
fuel_price2_df = pd.read_csv(fuel_price_csv)

fuel_price2_df.head()

,F,T,S,Value
0,1,2,1,0.1730
1,1,3,1,0.1746
2,1,4,1,0.1761
3,1,5,1,0.1777
4,2,2,1,0.1586


##### Skriver Cost of fuel til excel

In [ ]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    fuel_price2_df.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [ ]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [ ]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance

# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance
def calculate_emissions(row):
    distance =  distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['F']] * distance



# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)


##### MAX EMISSIONS PER TIME PERIOD

In [ ]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
# print(df_max_ems)

In [ ]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)
# print(df_max_ems.head())
# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,0,1650000
1,1,1650000
2,2,1650000
3,3,1650000
4,4,1650000
5,5,1650000


##### TIME USED

In [ ]:
# Create T_rs
combinations_r_s = itertools.product(r_values, s_values)
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route['Hours'].to_dict()
hours_route_dict = {key + 1: value for key, value in route_dict.items()}
print(hours_route_dict)
# lager egne distanser for elektrisk
# hours_route_el_dict = hours_route_el['Hours'].to_dict()
# hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}
# print(hours_route_el_dict)


def calculate_timeused(row):
    value =  hours_route_dict[row['R']]
    return value


df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

print(df_timeused.head())
# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 24.0, 2: 24.0, 3: 24.0, 4: 24.0, 5: 24.0, 6: 48.0, 7: 24.0, 8: 24.0, 9: 24.0, 10: 24.0, 11: 24.0, 12: 24.0, 13: 24.0, 14: 24.0, 15: 24.0, 16: 24.0, 17: 24.0, 18: 24.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 24.0, 25: 24.0, 26: 48.0, 27: 24.0, 28: 24.0, 29: 24.0, 30: 24.0, 31: 48.0, 32: 48.0, 33: 24.0, 34: 24.0, 35: 24.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 24.0, 41: 48.0, 42: 24.0, 43: 24.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 24.0, 50: 24.0, 51: 24.0, 52: 48.0, 53: 48.0, 54: 24.0, 55: 48.0, 56: 24.0, 57: 24.0, 58: 48.0, 59: 24.0, 60: 24.0, 61: 48.0, 62: 48.0, 63: 24.0, 64: 24.0, 65: 24.0, 66: 48.0, 67: 48.0, 68: 24.0, 69: 48.0, 70: 48.0, 71: 48.0, 72: 48.0, 73: 48.0, 74: 48.0, 75: 48.0, 76: 48.0, 77: 48.0, 78: 48.0, 79: 24.0, 80: 48.0, 81: 24.0, 82: 48.0, 83: 48.0, 84: 48.0, 85: 48.0, 86: 48.0, 87: 48.0, 88: 48.0, 89: 48.0, 90: 48.0, 91: 48.0, 92: 24.0, 93: 48.0, 94: 48.0, 95: 48.0, 96: 48.0, 97: 48.0, 98: 48.0, 99: 48.0, 100: 48.0, 101: 48

#### Installations in route

In [ ]:
import csv


# Open the existing Excel workbook
workbook = openpyxl.load_workbook(output_excel_path)
sheet = workbook["Installations in route"]

# Open the CSV file in text mode and read all lines
with open(routes_file, 'r') as file:
    lines = file.readlines()

# Add column headers if necessary
if not sheet['A1'].value:
    sheet['A1'] = "i"
    sheet['B1'] = "r"
    sheet['C1'] = "Value"

# Initialize the row index for Excel
row_index = 2

# Initialize an empty dictionary to store the platform mapping
platform_mapping = {}

# Path to your CSV file


# Open the CSV file and read the contents
with open(platform_demand, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        platform_mapping[platform] = len(platform_mapping) + 1

print(platform_mapping)


{'APT': 1, 'ASL': 2, 'DAB': 3, 'DSA': 4, 'DSS': 5, 'GFAGFBGFC': 6, 'KVB': 7, 'MID': 8, 'NLNVAL': 9, 'OSC': 10, 'OSE': 11, 'OSOVFB': 12, 'OSS': 13, 'STASTB': 14, 'STC': 15, 'TENTRB': 16, 'TEQTRC': 17, 'TRO': 18}


In [ ]:

platform_mapping_reverse = {v: k for k, v in platform_mapping.items()}

length = len(platform_mapping) +1

# Process each line in the CSV file
for idx, line in enumerate(lines, start=1):
    for i in range(1, length):
        sheet.cell(row=row_index, column=2, value=idx)
        sheet.cell(row=row_index, column=1, value=i)
        platform_name = platform_mapping_reverse.get(i, "")  # Get the platform name
        if platform_name in line:
            sheet.cell(row=row_index, column=3, value=1)
        else:
            sheet.cell(row=row_index, column=3, value=0)
        row_index += 1

workbook.save(excel_file)
print("Data has been successfully written.")

Data has been successfully written.


In [ ]:

csv_file = '../route_generation_mongstad/clustering/output_platforms_visits.csv'


visits_per_week = {}
with open(csv_file, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    i = 1
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        # Assign a number to each platform
        visits_per_week[i] = row[1]
        i += 1

i_values = range(1, len(platform_mapping)+1)
print(visits_per_week)

combinations_i_t = itertools.product(i_values, t_values)
df_demand = pd.DataFrame(combinations_i_t, columns=['I', 'T'])
# Iterate over rows in the DataFrame
df_demand['Value'] = df_demand['I'].map(lambda x: visits_per_week[x])



df_demand

{1: '2', 2: '3', 3: '4', 4: '3', 5: '3', 6: '4', 7: '2', 8: '2', 9: '3', 10: '2', 11: '5', 12: '4', 13: '3', 14: '4', 15: '3', 16: '4', 17: '4', 18: '2'}


,I,T,Value
0,1,0,2
1,1,1,2
2,1,2,2
3,1,3,2
4,1,4,2
...,...,...,...
103,18,1,2
104,18,2,2
105,18,3,2
106,18,4,2


In [ ]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand', index=False)